#Bracketology Notebook with D3.js and Python
I have spent a lot of time looking at March Madness through the lens of data analysis.  While I have written code to scrape the team and player statistics (which I elect to not share), one thing I found missing was a good way to display the results of my simulations.  While this was intended for March Madness, this D3.js and python code is universal enough to be extended to other situations where brackets are involved.  We create a bracket which is interactive (scrollable and zoomable), and display it in an Jupyter notebook.

This work was made significantly easiser by the following works:
* http://bl.ocks.org/jdarling/2503502
* http://thepowerrank.com/ncaa-tournament-predictions/
* http://bl.ocks.org/d3noob/8375092
* http://www.d3noob.org/2014/01/tree-diagrams-in-d3js_11.html
* http://www.robschmuecker.com/d3-js-drag-and-drop-zoomable-tree/
* http://bl.ocks.org/mbostock/2966094
* http://bl.ocks.org/robschmuecker/7926762
* http://bl.ocks.org/kanesee/5d6c48bffd4ea31201fb
* https://github.com/skariel/IPython_d3_js_demo/blob/master/d3_js_demo.ipynb

I'm still working on a universal way to make this a 2 way bracket


In [209]:
htmlcode = '''
<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="utf-8">

    <title>Collapsible Tree Example</title>

    <style>

 .node circle {
   fill: #fff;
   stroke: steelblue;
   stroke-width: 3px;
 }

 .node text { font: 12px sans-serif; }

 .link {
   fill: none;
   stroke: #ccc;
   stroke-width: 2px;
 }
 
    </style>

  </head>

  <body>

<!-- load the d3.js library --> 
<script src="http://d3js.org/d3.v3.min.js"></script>
 
<script>
var treeData = {myData};

// ************** Generate the tree diagram  *****************
var margin = {top: 20, right: 120, bottom: 20, left: 120},
 width = 1960 - margin.right - margin.left,
 height = 1500 - margin.top - margin.bottom;
 
var zoom = d3.behavior.zoom()
    .scaleExtent([1, 10])
    .on("zoom", zoomed);    
    
var i = 0;

var tree = d3.layout.tree()
 .size([height, width]);

var svg = d3.select("body").append("svg")
 .attr("width", width + margin.right + margin.left)
 .attr("height", height + margin.top + margin.bottom)
 .call(d3.behavior.zoom().on("zoom", function () {
    svg.attr("transform", "translate(" + d3.event.translate + ")" + " scale(" + d3.event.scale + ")")
  }))
 .append("g")
 .attr("transform", "translate(" + margin.left + "," + margin.top + ")");


root = treeData[0];
  
update(root);

function zoomed() {
  container.attr("transform", "translate(" + d3.event.translate + ")scale(" + d3.event.scale + ")");
}
   
function update(source) {

  // Compute the new tree layout.
  var nodes = tree.nodes(root).reverse(),
   links = tree.links(nodes);

  // Normalize for fixed-depth.
  nodes.forEach(function(d) { d.y = d.depth * 180; });

  // Declare the nodes
  var node = svg.selectAll("g.node")
   .data(nodes, function(d) { return d.id || (d.id = ++i); });

  // Enter the nodes.
  var nodeEnter = node.enter().append("g")
   .attr("class", "node")
   .attr("transform", function(d) { 
    return "translate(" + d.y + "," + d.x + ")"; });

  nodeEnter.append("circle")
      .attr("r", 4.5)
      .style("fill", function(d) { return d._children ? "lightsteelblue" : "#fff"; });

  nodeEnter.append("text")
   .attr("dy", -15)
   .attr("text-anchor", "middle")
   .text(function(d) { return d.name; })
   .style("fill-opacity", 1);

  // Declare the links
  var link = svg.selectAll("path.link")
   .data(links, function(d) { return d.target.id; });

  // Enter the links.
  link.enter().insert("path", "g")
   .attr("class", "link")
   .attr("d", elbow);

function elbow(d, i) {
  return "M" + d.source.y + "," + d.source.x
	  + "H" + ((d.source.y+d.target.y) / 2)
      + "V" + d.target.x + "H" + d.target.y ;
}   
   
   
}

</script>
 
  </body>
</html>
'''

#Python

In [210]:
import numpy as np

# Create the brackets.  The bracket2015 is an array of array's.  The first array is all the
# teams, second is the teams remaining after the first round, and so fourth until only 
# one team is left.

North = ['Duke',"UNF",'San Diego St.','St. Johns','Utah','SF Austin','Georgetown','E. Wash.','SMU','UCLA',
         'Iowa St.','UAB','Iowa','Davidson','Gonzaga','N. Dak. St.']
South = ['Kentucky','New Hampshire','Cincinnati','Purdue','W. Virginia','Buffalo','Maryland','Valparaiso','Butler',
         'Texas','Notre Dame','Northeastern','Wichita St.','Indiana','Kansas','N. Mex. St.']
East = ['Villanova','Lafayette','NC State','LSU','N. Iowa','Wyoming','Louisville','UC Irvine','Providence','Boise St.',
        'Oklahoma','Albany','Michigan St.',"Georgia",'Virginia','Belmont']
West = ['Wisconsin','Coast. Car.','Oregon',"Oklahoma St.",'Arkansas','Wofford','N. Carolina','Harvard','Xavier','BYU',
        'Baylor','Georgia St.','VCU','Ohio St.','Arizona','Texas So.']

#Set up the brackets 
bracket2015 = np.append([],North)
bracket2015 = np.append(bracket2015,South)
bracket2015 = np.append(bracket2015,East)
bracket2015 = np.append(bracket2015,West)
bracket2015 = [bracket2015]

#Randomly pick the winner.  Some function based on statistical analysis can go here too.

def winners(bracket2015):
    bracket = []
    an_array = np.random.randint(0,2,np.size(bracket2015[-1])/2)

    for value in an_array:
        if value:
            bracket = np.append(bracket,[True, False])
        else:
            bracket = np.append(bracket,[False, True])
    bracket2015.append(bracket2015[-1][bracket == True])
#    print(bracket2015)
    if np.size(bracket2015[-1])==1:
        return bracket2015
    else:
        return(winners(bracket2015))
    
bracket2015 = winners(bracket2015)
print(bracket2015)
        


[array(['Duke', 'UNF', 'San Diego St.', 'St. Johns', 'Utah', 'SF Austin',
       'Georgetown', 'E. Wash.', 'SMU', 'UCLA', 'Iowa St.', 'UAB', 'Iowa',
       'Davidson', 'Gonzaga', 'N. Dak. St.', 'Kentucky', 'New Hampshire',
       'Cincinnati', 'Purdue', 'W. Virginia', 'Buffalo', 'Maryland',
       'Valparaiso', 'Butler', 'Texas', 'Notre Dame', 'Northeastern',
       'Wichita St.', 'Indiana', 'Kansas', 'N. Mex. St.', 'Villanova',
       'Lafayette', 'NC State', 'LSU', 'N. Iowa', 'Wyoming', 'Louisville',
       'UC Irvine', 'Providence', 'Boise St.', 'Oklahoma', 'Albany',
       'Michigan St.', 'Georgia', 'Virginia', 'Belmont', 'Wisconsin',
       'Coast. Car.', 'Oregon', 'Oklahoma St.', 'Arkansas', 'Wofford',
       'N. Carolina', 'Harvard', 'Xavier', 'BYU', 'Baylor', 'Georgia St.',
       'VCU', 'Ohio St.', 'Arizona', 'Texas So.'], 
      dtype='|S13'), array(['UNF', 'St. Johns', 'Utah', 'E. Wash.', 'SMU', 'UAB', 'Davidson',
       'N. Dak. St.', 'Kentucky', 'Cincinnati', 'W. Virginia'

In [211]:
import numpy as np

# Create a tree bracket of arbitrary size, inputs only can be 2^x.
def treeMaker(size): 
    levels = np.log(size)/np.log(2)
    if np.log(size)%np.log(2) > 0:
        size = 2 ** int(levels)
    bracket = []     
    bracket = [{"name": team+1 , "parent" : "" , "children" : ""} for team in np.arange(size)]
    bracket = level(bracket)        
    return bracket

# Take an array of tree data and recursively combine it until all matches have been done
def level(bracket):
    newbracket = []
    for loop in np.arange(0, np.size(bracket),2):
        newbracket.append( {"name": "winner" , "parent" : "" , "children" : [bracket[loop],bracket[loop+1]]})
    if np.size(newbracket)>1:
        newbracket = level(newbracket)
    return newbracket

# change an array (or array of arrays) to a tree data form
def namedTree(bracket):
    newbracket = []
    for loop1 in bracket:
        levels = []
        for loop2 in loop1:
            levels.append( {"name": loop2 , "parent" : "" , "children" :""})
        newbracket.append(levels)
    return newbracket

# take an array of brackets and combine them
def combineData(arr):
    for loop1 in np.arange(1, np.size(arr)):
        for loop2 in np.arange(0, np.size(arr[loop1])):
            arr[loop1][loop2]['children'] = [arr[loop1-1][2*loop2], arr[loop1-1][2*loop2+1]]
            #print(arr[loop1][loop2])
    return arr[-1]

    

In [212]:
from IPython.display import IFrame
import re

def replace_all(txt,d):
    rep = dict((re.escape('{'+k+'}'), str(v)) for k, v in d.items())
    pattern = re.compile("|".join(rep.keys()))
    return pattern.sub(lambda m: rep[re.escape(m.group(0))], txt)    


count=0
def serve_html(s,w,h):
    import os
    global count
    count+=1
    fn= '__tmp'+str(os.getpid())+'_'+str(count)+'.html'
    with open(fn,'w') as f:
        f.write(s)
    return IFrame('files/'+fn,w,h)

def f2(w,h,data):
    d={'myData' : data}
#    with open('f2.template','r') as f:
#        s=f.read()
    s = htmlcode
    s = replace_all(s,d)        
    return serve_html(s,w+30,h+30)

In [213]:
myTree = namedTree(bracket2015)
bracketology = combineData(myTree)

In [214]:
am = treeMaker(64)  

In [215]:
f2(900,500,bracketology)

#Method #2

In [216]:
import IPython
from IPython.display import IFrame
from IPython.core.display import display_html, display_javascript, Javascript
import re

def replace_all(txt,d):
    rep = dict((re.escape('{'+k+'}'), str(v)) for k, v in d.items())
    pattern = re.compile("|".join(rep.keys()))
    return pattern.sub(lambda m: rep[re.escape(m.group(0))], txt)    


def serve_html(s,w,h):
    display_html(IPython.core.display.HTML(data=s))
    #display_javascript( Javascript(data=test_d3_js, lib= js_libs))

def create(w,h,data):
    d={'myData' : data}
    js_libs = ['http://rawgithub.com/mbostock/d3/master/d3.min.js']
    s = html_src2
    s = replace_all(s,d)        
    serve_html(s,w+30,h+30)


In [217]:
html_src = """
                <h2>Simple D3 plot</h2>
                <style>

                 .node circle {
                   fill: #fff;
                   stroke: steelblue;
                   stroke-width: 3px;
                     }

                .node text { font: 12px sans-serif; }

                .link {
                   fill: none;
                   stroke: #ccc;
                   stroke-width: 2px;
                 }
 
                </style> 
                <div id="chart"></div>
           """
#add style     <style>
#
# .node circle {
#   fill: #fff;
#   stroke: steelblue;
#   stroke-width: 3px;
# }

# .node text { font: 12px sans-serif; }

# .link {
#   fill: none;
#   stroke: #ccc;
#   stroke-width: 2px;
# }
 
#    </style> 


test_d3_js_2 = """

                    var treeData = {myData};

// ************** Generate the tree diagram  *****************
var margin = {top: 20, right: 120, bottom: 20, left: 120},
 width = 1960 - margin.right - margin.left,
 height = 1500 - margin.top - margin.bottom;
 
var zoom = d3.behavior.zoom()
    .scaleExtent([1, 10])
    .on("zoom", zoomed);    
    
var i = 0;

var tree = d3.layout.tree()
 .size([height, width]);

var svg = d3.select('#chart').append("svg")
 .attr("width", width + margin.right + margin.left)
 .attr("height", height + margin.top + margin.bottom)
 .call(d3.behavior.zoom().on("zoom", function () {
    svg.attr("transform", "translate(" + d3.event.translate + ")" + " scale(" + d3.event.scale + ")")
  }))
 .append("g")
 .attr("transform", "translate(" + margin.left + "," + margin.top + ")");


var root =  treeData[0];
  
update(root);

function zoomed() {
  container.attr("transform", "translate(" + d3.event.translate + ")scale(" + d3.event.scale + ")");
}
   
function update(source) {

  // Compute the new tree layout.
  var nodes = tree.nodes(root).reverse(),
   links = tree.links(nodes);

  // Normalize for fixed-depth.
  nodes.forEach(function(d) { d.y = d.depth * 180; });

  // Declare the nodes
  var node = svg.selectAll("g.node")
   .data(nodes, function(d) { return d.id || (d.id = ++i); });

  // Enter the nodes.
  var nodeEnter = node.enter().append("g")
   .attr("class", "node")
   .attr("transform", function(d) { 
    return "translate(" + d.y + "," + d.x + ")"; });

  nodeEnter.append("circle")
      .attr("r", 4.5)
      .style("fill", function(d) { return d._children ? "lightsteelblue" : "#fff"; });

  nodeEnter.append("text")
   .attr("dy", -15)
   .attr("text-anchor", "middle")
   .text(function(d) { return d.name; })
   .style("fill-opacity", 1);

  // Declare the links
  var link = svg.selectAll("path.link")
   .data(links, function(d) { return d.target.id; });

  // Enter the links.
  link.enter().insert("path", "g")
   .attr("class", "link")
   .attr("d", elbow);

function elbow(d, i) {
  return "M" + d.source.y + "," + d.source.x
	  + "H" + ((d.source.y+d.target.y) / 2)
      + "V" + d.target.x + "H" + d.target.y ;
}   
   
   
};    
             """
 
js_libs = ["http://d3js.org/d3.v3.min.js"]

import IPython 
from IPython.core.display import display_html, display_javascript, Javascript


d={'myData' : bracketology}
test_d3_js_2 = replace_all(test_d3_js_2,d)        

display_html(IPython.core.display.HTML(data=html_src))
display_javascript( Javascript(data=test_d3_js_2, lib= js_libs))

Simple D3 plot